In [1]:
from google.colab import drive  #in case test data is on the drive
drive.mount('/content/drive')

Mounted at /content/drive


Cell #1 Input test directory

In [2]:
#--------------------------------------------------------------------- (*)required to run this cell---------------------------------------------------------------------

#---------------------------note: the file includes all the code. training part is commented. Since the code was developed for training naming might be confusing.
#---------------------------Instructions what needs to be changed and run is give in comments
import pickle                                                     #read input and output languages  

train_input = pickle.load(open('/content/drive/MyDrive/project2.0/DS_5_train_input','rb'))    #--------------------------> change dataset directory 
train_output = pickle.load(open('/content/drive/MyDrive/project2.0/DS_5_train_output','rb'))  #--------------------------> change dataset directory 

print(train_input[2])
print(train_output[2])

a k c f a d a g b e a h c d a h a k a k b f b e b e b d c d a e c d b f 
c f b e c d b f b e a k g h b e a k i j b d c d a h k l m c d a e ee a h f ed ef a g e eg b f a d eh ei a k d ej 


In [4]:
#--------------------------------------------------------------------- (*)required to run this cell---------------------------------------------------------------------
import tensorflow as tf                                                
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

vocab_size = 500                                                     # For time limit assume 500 words
sequence_length = max([len(train_output[i].split()) for i in range(len(train_output))])  #maximum length of a sentence in the dataset

print("Maximum output language text length: "+str(sequence_length))
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1
)

source_vectorization.adapt(train_input)           
target_vectorization.adapt(train_output)

Maximum output language text length: 95


In [5]:
#--------------------------------------------------------------------- (*)required to run this cell---------------------------------------------------------------------
text_pairs = list()                                                          #creating list of text pairs
for n in range(len(train_input)):
    text_pairs.append((train_input[n], "[start] "+train_output[n]+" [end]")) # adding [start] and [end] to the translatin 

'''
import random
random.shuffle(text_pairs)                                                    #suffle and divide into train, val and test
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
'''
test_pairs = text_pairs

In [6]:
'''
batch_size = 64

def format_dataset(lang1, lang2):
    lang1 = source_vectorization(lang1)
    lang2 = target_vectorization(lang2)
    return ({
        "language1": lang1,
        "language2": lang2[:, :-1],
    }, lang2[:, 1:])

def make_dataset(pairs):
    lang1_texts, lang2_texts = zip(*pairs)
    lang1_texts = list(lang1_texts)
    lang2_texts = list(lang2_texts)
    dataset = tf.data.Dataset.from_tensor_slices((lang1_texts, lang2_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)       #creating training and pal pairs for training the model in batches 
val_ds = make_dataset(val_pairs)
'''

'\nbatch_size = 64\n\ndef format_dataset(lang1, lang2):\n    lang1 = source_vectorization(lang1)\n    lang2 = target_vectorization(lang2)\n    return ({\n        "language1": lang1,\n        "language2": lang2[:, :-1],\n    }, lang2[:, 1:])\n\ndef make_dataset(pairs):\n    lang1_texts, lang2_texts = zip(*pairs)\n    lang1_texts = list(lang1_texts)\n    lang2_texts = list(lang2_texts)\n    dataset = tf.data.Dataset.from_tensor_slices((lang1_texts, lang2_texts))\n    dataset = dataset.batch(batch_size)\n    dataset = dataset.map(format_dataset, num_parallel_calls=4)\n    return dataset.shuffle(2048).prefetch(16).cache()\n\ntrain_ds = make_dataset(train_pairs)       #creating training and pal pairs for training the model in batches \nval_ds = make_dataset(val_pairs)\n'

In [7]:
'''
for inputs, targets in train_ds.take(1):                            #checking out dataset shape (not required for testing)
    print(f"inputs['language1'].shape: {inputs['language1'].shape}")
    print(f"inputs['language2'].shape: {inputs['language2'].shape}")
    print(f"targets.shape: {targets.shape}
'''

'\nfor inputs, targets in train_ds.take(1):                            #checking out dataset shape (not required for testing)\n    print(f"inputs[\'language1\'].shape: {inputs[\'language1\'].shape}")\n    print(f"inputs[\'language2\'].shape: {inputs[\'language2\'].shape}")\n    print(f"targets.shape: {targets.shape}\n'

In [8]:
'''
embed_dim = 256                            #building up a model - takes first language a an input
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="language1")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)
'''

'\nembed_dim = 256                            #building up a model - takes first language a an input\nlatent_dim = 1024\n\nsource = keras.Input(shape=(None,), dtype="int64", name="language1")\nx = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)\nencoded_source = layers.Bidirectional(\n    layers.GRU(latent_dim), merge_mode="sum")(x)\n'

In [9]:
'''
past_target = keras.Input(shape=(None,), dtype="int64", name="language2") #branch that takes target language as an input 
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)  #writing the sentence in 0s and 1s
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)
'''

'\npast_target = keras.Input(shape=(None,), dtype="int64", name="language2") #branch that takes target language as an input \nx = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)  #writing the sentence in 0s and 1s\ndecoder_gru = layers.GRU(latent_dim, return_sequences=True)\nx = decoder_gru(x, initial_state=encoded_source)\nx = layers.Dropout(0.5)(x)\ntarget_next_step = layers.Dense(vocab_size, activation="softmax")(x)\nseq2seq_rnn = keras.Model([source, past_target], target_next_step)\n'

Cell #2 Input model durectory

In [13]:
#--------------------------------------------------------------------- (*)required to run this cell---------------------------------------------------------------------
'''
seq2seq_rnn.compile(                                                                                 #choosing model's accuracy and loss
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
'''
seq2seq_rnn = keras.models.load_model('/content/drive/MyDrive/project2.0/project2_5.keras')      #------------------------------------->model directory
#callbacks = [keras.callbacks.ModelCheckpoint("project2_5.keras",                                     #saving updated model 
#                                    save_best_only=True)]                                            #training the model 
#seq2seq_rnn.fit(train_ds, epochs=5, validation_data=val_ds, callbacks = callbacks)

In [14]:
#--------------------------------------------------------------------- (*)required to run this cell---------------------------------------------------------------------
from inspect import CORO_CREATED
import numpy as np
lang2_vocab = target_vectorization.get_vocabulary()
lang2_index_lookup = dict(zip(range(len(lang2_vocab)), lang2_vocab))
max_decoded_sentence_length = 95

def decode_sequence(input_sentence):                                   #returns translated sentence for each input 
    tokenized_input_sentence = source_vectorization([input_sentence])  #write words as numbers
    decoded_sentence = "[start]"                                       #initialize the target sentence 
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence]) #write words as numbers
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence]) #predicts next token given the current translation and input
        sampled_token_index = np.argmax(next_token_predictions[0, i, :]) #numerical index of the predicted word
        sampled_token = lang2_index_lookup[sampled_token_index] #predicted word 
        decoded_sentence += " " + sampled_token  #add to translation 
        if sampled_token == "[end]": 
            break
    return decoded_sentence

correct = 0.0
total = 0.0
#number_of_tests = 5 #in case of early break uncomment if statement
for  i,  pair in enumerate(test_pairs):
    ground_truth = pair[1].split()                  #true translation 
    prediction = decode_sequence(pair[0]).split()   #predicted translation 
    total+=len(ground_truth)-2                      #total number of words in true translation 
    for j in range(len(ground_truth)-1):
        if len(prediction) > j and j!=0:
            if ground_truth[j] == prediction[j]:
                correct+=1                          #correctly translated words in prediction 
    print("ground truth: "+str(ground_truth[1:len(ground_truth)-1])) #printing out true translation and prediction 
    print("prediction: "+str(prediction[1:len(prediction)-1]))
    print("accuracy for "+str(i+1)+" samples: "+str(correct/(1.0*total))) #updated accuracy after one translation
    #if i == number_of_tests-1:
        #break
print("accuracy: "+str(correct/(1.0*total))) #total accuracy

ground truth: ['b', 'f', 'c', 'f', 'b', 'f', 'c', 'd', 'a', 'j', 'e', 'f', 'g', 'c', 'e', 'b', 'g', 'a', 'k', 'i', 'j', 'b', 'd', 'b', 'f', 'a', 'k', 'l', 'm', 'b', 'f', 'b', 'd', 'a', 'h', 'ed', 'ee', 'ef', 'a', 'k', 'k', 'eg', 'a', 'k', 'h', 'eh', 'a', 'e', 'ei', 'c', 'd', 'a', 'f', 'ej', 'ek', 'a', 'g', 'd', 'el']
prediction: ['b', 'f', 'c', 'f', 'b', 'f', 'c', 'd', 'a', 'j', 'e', 'f', 'g', 'c', 'e', 'b', 'g', 'b', 'd', 'b', 'g', 'a', 'k', 'k', 'l', 'a', 'k', 'j', 'm', 'b', 'f', 'a', 'k', 'ed', 'ee', 'b', 'd', 'a', 'd', 'ef', 'eg', 'a', 'f', 'i', 'eh', 'a', 'k', 'h', 'ei', 'a', 'e', 'ej', 'a', 'k', 'd', 'ek']
accuracy for 1samples: 0.3898305084745763
ground truth: ['b', 'e', 'c', 'd', 'b', 'g', 'c', 'g', 'a', 'i', 'e', 'f', 'g', 'a', 'f', 'd', 'h', 'c', 'e', 'c', 'f', 'c', 'f', 'a', 'e', 'l', 'c', 'd', 'a', 'i', 'k', 'm', 'ed', 'a', 'i', 'i', 'j', 'ee', 'a', 'e', 'ef', 'a', 'e', 'eg', 'a', 'e', 'eh', 'c', 'd', 'a', 'k', 'ei', 'ej']
prediction: ['b', 'e', 'c', 'd', 'b', 'g', 'c', 'g'